In [4]:
# This changes the current directory to the base saga directory - make sure to run this first!
# This is necessary to be able to import the py files and use the right directories,
# while keeping all the notebooks in their own directory.
import os
import sys

if 'saga_base_dir' not in locals():
    saga_base_dir = os.path.abspath('..')

if saga_base_dir not in sys.path:
    os.chdir(saga_base_dir)

In [55]:
from __future__ import print_function, division

import numpy as np

from astropy import units as u
from astropy import constants as cnst
from astropy import table

In [6]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rcParams

rcParams['image.interpolation'] = 'none'
rcParams['figure.figsize'] = (16, 10)
plt.rcParams['image.cmap'] = 'viridis'

# Find all of the sats that are in SAGA hosts

In [46]:
t = table.Table.read('SAGADropbox/data/saga_spectra_clean.fits.gz')

In [45]:
sagamasks = ~((t['TELNAME']=='SDSS')|(t['TELNAME']=='NSA')|(t['TELNAME']=='GAMA'))
sagaids = np.unique(t['HOST_NSAID'][sagamasks])
sagaspectra = t[np.in1d(t['HOST_NSAID'], sagaids)]

sats = sagaspectra[sagaspectra['SATS']==1]
sats

OBJID,RA,DEC,PHOTPTYPE,PHOT_SG,FLAGS,clean,SATURATED,BAD_COUNTS_ERROR,BINNED1,TOO_FEW_GOOD_DETECTIONS,u,g,r,i,z,u_err,g_err,r_err,i_err,z_err,MODELMAGERR_U,MODELMAGERR_G,MODELMAGERR_R,MODELMAGERR_I,MODELMAGERR_Z,EXTINCTION_U,EXTINCTION_G,EXTINCTION_R,EXTINCTION_I,EXTINCTION_Z,DERED_U,DERED_G,DERED_R,DERED_I,DERED_Z,PETRORAD_U,PETRORAD_G,PETRORAD_R,PETRORAD_I,PETRORAD_Z,PETRORADERR_U,PETRORADERR_G,PETRORADERR_R,PETRORADERR_I,PETRORADERR_Z,DEVRAD_U,DEVRADERR_U,DEVRAD_G,DEVRADERR_G,DEVRAD_R,DEVRADERR_R,DEVRAD_I,DEVRADERR_I,DEVRAD_Z,DEVRADERR_Z,DEVAB_U,DEVAB_G,DEVAB_R,DEVAB_I,DEVAB_Z,CMODELMAG_U,CMODELMAGERR_U,CMODELMAG_G,CMODELMAGERR_G,CMODELMAG_R,CMODELMAGERR_R,CMODELMAG_I,CMODELMAGERR_I,CMODELMAG_Z,CMODELMAGERR_Z,PSFMAG_U,PSFMAGERR_U,PSFMAG_G,PSFMAGERR_G,PSFMAG_R,PSFMAGERR_R,PSFMAG_I,PSFMAGERR_I,PSFMAG_Z,PSFMAGERR_Z,FIBERMAG_U,FIBERMAGERR_U,FIBERMAG_G,FIBERMAGERR_G,FIBERMAG_R,FIBERMAGERR_R,FIBERMAG_I,FIBERMAGERR_I,FIBERMAG_Z,FIBERMAGERR_Z,FRACDEV_U,FRACDEV_G,FRACDEV_R,FRACDEV_I,FRACDEV_Z,Q_U,U_U,Q_G,U_G,Q_R,U_R,Q_I,U_I,Q_Z,U_Z,EXPAB_U,EXPRAD_U,EXPPHI_U,EXPAB_G,EXPRAD_G,EXPPHI_G,EXPAB_R,EXPRAD_R,EXPPHI_R,EXPAB_I,EXPRAD_I,EXPPHI_I,EXPAB_Z,EXPRAD_Z,EXPPHI_Z,FIBER2MAG_R,FIBER2MAGERR_R,EXPMAG_R,EXPMAGERR_R,PETROR50_R,PETROR90_R,PETROMAG_R,SB_EXP_R,SB_PETRO_R,J,JERR,H,HERR,K,KERR,SPEC_Z,SPEC_Z_ERR,SPEC_Z_WARN,PHOTOZ,PHOTOZ_ERR,W1,W1ERR,W2,W2ERR,HOST_RA,HOST_DEC,HOST_DIST,HOST_VHOST,HOST_MK,HOST_NSAID,HOST_FLAG,HOST_SAGA_NAME,RHOST_ARCM,RHOST_KPC,OBJ_NSAID,SATS,PROBABILITY_CLASS1,RESCALED_PROBABILITY_CLASS1,REMOVE,TELNAME,MASKNAME,ZQUALITY,SPEC_CLASS,SPECOBJID,SPEC_REPEAT
int64,float64,float64,int16,str40,int64,int32,int64,int64,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,str48,float64,float64,int64,int64,float64,float64,int64,str6,str48,int64,str2,str48,str48
1237651735757259099,180.636494306,1.95802947978,3,GALAXY,281543964623104,0,0,0,268435456,281474976710656,26.9357,19.2637,18.6728,18.9803,26.7732,3.48973,0.0899796,0.0839212,0.173803,0.658678,3.48973,0.0899796,0.0839212,0.173803,0.658678,0.115032,0.0846394,0.0613876,0.0465483,0.0330033,26.8206,19.1791,18.6114,18.9337,26.7402,2.97098,2.97006,258.374,2.96948,2.97022,-1000.0,-1000.0,-1000.0,-1000.0,-1000.0,2.12566,3.65126,29.6753,2.31745,27.1824,8.31657,29.6695,2.53441,0.00358716,27.1103,0.05,0.623367,0.999923,0.952344,0.0501283,21.8828,0.386924,19.1192,0.0916567,17.2244,0.0721443,18.0551,0.166074,22.1379,0.697607,22.5648,0.410591,22.7915,0.178714,22.782,0.271717,23.0981,0.472571,22.0059,0.635566,23.3443,0.69363,22.7732,0.188145,22.6497,0.264579,22.8432,0.479258,22.1462,0.862093,1.0,0.337854,0.301263,0.0654641,1.0,-0.191439,0.930425,-0.0791527,0.00684313,-0.136734,0.142043,0.126804,0.243481,-0.424756,-3.02252,0.05,3.41202,63.378,0.344957,17.0706,153.516,0.374638,57.5764,150.554,0.679342,24.642,146.598,0.300015,0.000382415,90.4881,23.3621,0.323969,16.9897,0.0668231,3.80096,5.55442,20.4335,27.7863323159,25.328416084

In [110]:
palomar_A = 1.76e5 * u.cm**2

# CWI

From Jeremy:

Since you have spectra already, this should be pretty straight-forward. I use 20 minute exposures. Sometimes I combine two 20 minute exposures. The staff recommends sticking to this observing strategy. For the 20 minute exposures, the typical noise per pixel in a given spaxel is about 7x10^-18 erg/s/cm^2 (+/- 1x10^-18 erg/s/cm^2). This is pretty consistent across the entire bandwidth and across the CCD. I have found you can measure line centers down to a S/N of 5. I have not done mosaics but I know others have. The spatial resolution is about 2.7" by whatever your seeing is. You can rotate the FOV 90 degrees and co-add to get seeing limited data but rotation is not a simple task. The FOV is 60" by 40" but the edges of the field are a little wonky. The nod-and-shuffle mask limits the bandwidth range but is fine for H-alpha and both NII lines.

In [116]:
noise = 7e-18 * u.erg/u.s*u.cm**-2

def mag_to_snr(mag, dellamb, lamb):
    num = palomar_A * dellamb * (mag*u.ABmag).physical
    den = cnst.h * lamb
    return (num/den/noise).decompose()
mag_to_signal(sats['FIBER2MAG_R'], 1*u.angstrom, 6563*u.angstrom)

<Quantity [  0.06642015,  0.75046402,  2.325243  ,  0.53888935,
             0.8946681 ,  0.14583386,  0.09362393,  0.3214168 ,
             0.75702339,  0.20041418,  0.51558721,  0.1318621 ,
             2.48143744,  0.76061094,  0.77778667,  5.24280167,
             1.18105042, 46.94685364,  4.28348494,  0.08692928,
             1.81869495,  2.14886451,  4.66594887,  0.39642385,
             0.16374445,  1.14741862,  0.33538094,  0.31246436,
             7.17035532,  2.39514327,  0.56655043,  0.1776437 ,
             2.0448513 ,  2.39981318,  2.80552244,  0.19174385,
            17.6655426 ,  0.36412913,  3.59615183,  4.43133259,
             0.5036605 ,  0.14256053,  0.25674385, 10.12567425,
             0.41092703,  0.35267577,  2.11215377,  1.65712404,
             0.81871659] 1 / s>

# DBSpec 

Following http://www.astro.caltech.edu/~enk/dbsp_throughput/, but just assuming a straight ~20% throughput, which is good for ~Halpha for most gratings

In [94]:
m_to_nphot(10, 1*u.second, 1*u.angstrom, 6563*u.angstrom)

<Quantity 2938.896338544408>

In [95]:
def m_to_nphot(mag, texp, dellamb, lamb, throughput=.2):
    num = texp * palomar_A * dellamb * (mag*u.ABmag).physical
    den = cnst.h * lamb
    return (throughput*num/den).decompose()
m_to_nphot(20, 1*u.second, 1*u.angstrom, 6563*u.angstrom)

<Quantity 0.2938896338544408>

In [104]:
# let's just say 2" is typical seeing
countrate = m_to_nphot(sats['FIBER2MAG_R'], 1*u.second, 1*u.angstrom, 6563*u.angstrom)/u.second

# assume we need ~100 counts per/angstrom for a decent centroid
time_needed = 100/ countrate
time_needed

<Quantity [ 7527.83544922,  666.25445557,  215.03125   ,  927.83422852,
             558.86645508, 3428.55908203, 5340.51513672, 1555.61242676,
             660.48156738, 2494.83325195,  969.76794434, 3791.83984375,
             201.49609375,  657.36633301,  642.84979248,   95.36885071,
             423.35195923,   10.65034103,  116.72737885, 5751.80273438,
             274.92242432,  232.68101501,  107.1593399 , 1261.27636719,
            3053.53857422,  435.76074219, 1490.84191895, 1600.18225098,
              69.73154449,  208.75578308,  882.53387451, 2814.62280273,
             244.51655579,  208.3495636 ,  178.21990967, 2607.64550781,
              28.30368614, 1373.1394043 ,  139.0375061 ,  112.83287048,
             992.73223877, 3507.2824707 , 1947.46643066,   49.37942886,
            1216.76098633, 1417.73278809,  236.72515869,  301.72753906,
             610.7119751 ] s>

In [106]:
np.sum(time_needed).to(u.hour)

<Quantity 17.678226996527776 h>